In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import pandas as pd
import os
import torch

In [ ]:
# def csv_to_coco(csv_path, meta_csv, output_json):
#     df = pd.read_csv(csv_path)
#     meta_df = pd.read_csv(meta_csv)
#     filtered_df = df[df["image_id"].isin(meta_df["image_id"])]

#     class_map = {"ulcer": 0, "polyp": 1, "cancer": 2}
#     coco_json = {"images": [], "annotations": [], "categories": []}

#     for class_name, class_id in class_map.items():
#         coco_json["categories"].append({"id": class_id, "name": class_name})

#     image_id_map = {}
#     annotation_id = 1

#     for _, row in filtered_df.iterrows():
#         image_id = row["image_id"]
#         image_name = row["image_id"]

#         if image_id not in image_id_map:
#             image_id_map[image_id] = image_name
#             coco_json["images"].append({
#                 "id": image_id,
#                 "width": row["width"],
#                 "height": row["height"],
#                 "file_name": image_name
#             })

#         x_min, y_min, x_max, y_max = row["x_min"], row["y_min"], row["x_max"], row["y_max"]
#         bbox = [x_min, y_min, x_max - x_min, y_max - y_min]

#         coco_json["annotations"].append({
#             "id": annotation_id,
#             "image_id": image_id,
#             "category_id": class_map[row["lesion"]],
#             "bbox": bbox,
#             "area": bbox[2] * bbox[3],
#             "iscrowd": 0
#         })
#         annotation_id += 1

#     with open(output_json, "w") as f:
#         json.dump(coco_json, f, indent=4)

#     print(f"COCO JSON saved: {output_json}")

# base_path = r"/content/drive/MyDrive/Endoscope Detection/colonoscopy"
# csv_path = r"/content/drive/MyDrive/Endoscope Detection/colonoscopy/resize_colonoscopy.csv"

# meta_paths = {
#     "train": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/train_meta.csv",
#     "val": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/val_meta.csv",
#     "test": "/content/drive/MyDrive/Endoscope Detection/colonoscopy/test_meta.csv",
# }

# json_paths = {
#     "train": os.path.join(base_path, "Dataset", "train_coco.json"),
#     "val": os.path.join(base_path, "Dataset", "val_coco.json"),
#     "test": os.path.join(base_path, "Dataset", "test_coco.json"),
# }

# for split in ["train", "val", "test"]:
#     csv_to_coco(csv_path, meta_paths[split], json_paths[split])


In [3]:
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.4 MB/s eta 0:00:00


In [4]:
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
import json

In [5]:
from detectron2.data import DatasetCatalog, MetadataCatalog
import json

def register_coco_dataset(name, json_file, image_root):
    DatasetCatalog.register(
        name,
        lambda: load_coco_json(json_file, image_root)
    )
    MetadataCatalog.get(name).set(thing_classes=["ulcer", "polyp","cancer"])

def load_coco_json(json_file, image_root):
    with open(json_file, "r") as f:
        coco_data = json.load(f)

    dataset_dicts = []
    for image_info in coco_data['images']:
        record = {}
        record['file_name'] = os.path.join(image_root, image_info['file_name'])
        record['image_id'] = image_info['id']
        record['height'] = image_info['height']
        record['width'] = image_info['width']

        annotations = []
        for ann in coco_data['annotations']:
            if ann['image_id'] == image_info['id']:
                annotations.append({
                    "bbox": ann['bbox'],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": ann['category_id']
                })

        record['annotations'] = annotations
        dataset_dicts.append(record)
    return dataset_dicts


image_folders = {
    "train": r"/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/train",
    "val": r"/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/val",
    "test": r"/content/drive/MyDrive/Endoscope Detection/colonoscopy/Dataset/images/test"
}

base_path = r"/content/drive/MyDrive/Endoscope Detection/colonoscopy"
json_paths = {
    "train": os.path.join(base_path, "Dataset", "train_coco.json"),
    "val": os.path.join(base_path, "Dataset", "val_coco.json"),
    "test": os.path.join(base_path, "Dataset", "test_coco.json"),
}

for split in ["train", "val", "test"]:
    register_coco_dataset(f"colonoscopy_{split}", json_paths[split], image_folders[split])


In [6]:
!pip install albumentations

In [7]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_train_loader, build_detection_test_loader

In [ ]:
!wget -O /content/drive/MyDrive/Endoscope/Detection/colonoscopy/Base-RCNN-FPN.yaml https://raw.githubusercontent.com/facebookresearch/detectron2/main/configs/Base-RCNN-FPN.yaml


--2025-02-19 05:29:12--  https://raw.githubusercontent.com/facebookresearch/detectron2/main/configs/Base-RCNN-FPN.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1318 (1.3K) [text/plain]
Saving to: ‘/content/drive/MyDrive/Endoscope/Detection/colonoscopy/Base-RCNN-FPN.yaml’

/content/drive/MyDr 100%[===================>]   1.29K  --.-KB/s    in 0s      

2025-02-19 05:29:12 (51.1 MB/s) - ‘/content/drive/MyDrive/Endoscope/Detection/colonoscopy/Base-RCNN-FPN.yaml’ saved [1318/1318]



In [32]:
from detectron2.data import DatasetCatalog, MetadataCatalog, DatasetMapper
from detectron2.data.transforms import (RandomFlip, RandomRotation, RandomContrast, RandomBrightness, RandomLighting)
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
import os

In [33]:
def adjust_augmentation(cfg):
    augmentations = [
        RandomFlip(prob=0.4, horizontal=True, vertical=False),
        RandomContrast(intensity_min = 0.8, intensity_max=1.2),
        RandomBrightness(intensity_min=0.5, intensity_max=1.5)
    ]

    mapper = DatasetMapper(cfg, is_train=True, augmentations=augmentations)

    return build_detection_train_loader(cfg, mapper=mapper)

In [34]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return adjust_augmentation(cfg)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = cfg.OUTPUT_DIR
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

In [36]:
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Endoscope Detection/colonoscopy/config.yaml")

cfg.OUTPUT_DIR = "/content/drive/MyDrive/Endoscope Detection/colonoscopy/Detectron2/experiment/v2"
cfg.DATASETS.TRAIN = ("colonoscopy_train",)
cfg.DATASETS.TEST = ("colonoscopy_val",)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[02/20 06:38:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
  fc1000.{bias, weight}
  stem.conv1.bias


[02/20 06:38:56 d2.engine.train_loop]: Starting training from iteration 0
[02/20 06:38:59 d2.utils.events]:  eta: 0:16:56  iter: 19  total_loss: 3.754  loss_cls: 3.029  loss_box_reg: 0.02571  loss_rpn_cls: 0.665  loss_rpn_loc: 0.009196    time: 0.1022  last_time: 0.1017  data_time: 0.0100  last_data_time: 0.0035   lr: 4.9952e-05  max_mem: 1868M
[02/20 06:39:01 d2.utils.events]:  eta: 0:17:04  iter: 39  total_loss: 0.7772  loss_cls: 0.1708  loss_box_reg: 0.03841  loss_rpn_cls: 0.5753  loss_rpn_loc: 0.01081    time: 0.1035  last_time: 0.1032  data_time: 0.0037  last_data_time: 0.0037   lr: 9.9902e-05  max_mem: 1868M
[02/20 06:39:03 d2.utils.events]:  eta: 0:17:14  iter: 59  total_loss: 0.6986  loss_cls: 0.2292  loss_box_reg: 0.05555  loss_rpn_cls: 0.3841  loss_rpn_loc: 0.01482    time: 0.1101  last_time: 0.1174  data_time: 0.0052  last_data_time: 0.0041   lr: 0.00014985  max_mem: 1868M
[02/20 06:39:05 d2.utils.events]:  eta: 0:17:34  iter: 79  total_loss: 0.4997  loss_cls: 0.2164  loss_b